<div style="direction:rtl;line-height:300%;">
	<font face="XB Zar" size=5>
		<div align=center>
			<font face="IranNastaliq" size=30>
				<p></p>
				<p></p>
به نام خدا
				<p></p>
			</font>
			<p></p>
			<font color=blue>
پروژه ی filler
            </font>
			<br />
			<br />
دی ماه 97
		</div>
		<hr/>
		<font color=red size=6>
			<br />
			<div align=center>
decoders
            </div>        
		</font>
		<br />
		<div align=center>
 مهدی غزنوی
        </div>
		<hr />
		<style type="text/css" scoped>
        p{
        border: 1px solid #a2a9b1;background-color: #f8f9fa;display: inline-block;
        };
        </style>
	</font>
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        ما دو ابزار اساسی برای اجرای عملیات بر روی sequenceها به طور dynamic در tensorflow داریم.
        یکی <a href="https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn">dynamic rnn</a> و دیگری <a href="https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode">dynamic decode</a>.
        در ادامه قابلیت های هرکدام از این دو را مورد بررسی قرار می دهیم.
    </font> 
</div>

<div style="direction:ltr;line-height:300%;">
    <font face="XB Zar" size=6>
        dynamic decode
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
decoderها ابزار اصلی استفاده از dynamic decode هستند. dynamic decode به عنوان ورودی یک decoder دریافت می کند، سپس تابع initialize آن را صدا می زند و بعد از آن در هر مرحله تابع step دیکودر توسط dynamic decode صدا زده می شود.
<br/>
پس از پایان پروسه ی decoding، خروجی dynamic decode به صورت 
    <br/>(final_outputs, final_state, final_sequence_length) 
    <br/>
    خواهد بود. 
    </font> 
</div>

In [ ]:
tf.contrib.seq2seq.dynamic_decode(
    decoder,
    output_time_major=False,
    impute_finished=False,
    maximum_iterations=None,
    parallel_iterations=32,
    swap_memory=False,
    scope=None
)

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        در ورودی تابع اگر impute_finished را برابر True قرار دهیم پس از اتمام decoding یک sample تا زمان اتمام decoding بقیه sampleهای batch خروجی را برابر 0 و state را برابر آخرین state پیش از اتمام sample قرار می دهد.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        final_outputs یک tensor با بعد r+1 است در صورتی که بعد خروجی هر مرحله برابر r باشد. در واقع final_outputs، تنسور stack شده ی tensorArrayای است که عضو iام آن خروجی مرحله ی iام decoding است.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        حال به خود decoderها می پردازیم.
        <br/>
        interfaceی به نام Decoder در tensorflow وجود دارد که کلاس هایی که آن را implement می کنند می بایست سه تابع زیر را پیاده سازی کنند.
        <ul style="direction:ltr">
            <li> initialize(name=None)</li>
            <li> step(time, inputs, state, name=None)</li>
            <li> finalize(outputs, final_state, sequence_length) </li>
        </ul>
  
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        initialize:<br/>
        پیش از شروع decoding صدا زده می شود و ورودی و state اولیه  که در مرحله ی بعد به تابع step  داده می شود را به عنوان خروجی باز می گرداند. همچنین به عنوان خروجی مشخص می کند که آیا decoding به اتمام رسیده(
        طول خرجی 0 است) یا خیر.
        <br/>
        خروجی آن به صورت رو به رو است:
            (finished, initial_inputs, initial_state)
        
    </font> 
</div>

<div id="decoder_output" style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        step:<br/>
        در هر مرحله ی decoding صدا زده می شود و خروجی آن مرحله، ورودی مرحله ی بعدی، state بعدی و اینکه آیا decoding به پایان رسیده است یا خیر را مشخص می کند.
        <br/>
        خروجی این تابع به صورت رو به رو است: (outputs, next_state, next_inputs, finished)
        
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        finalize:<br/>
        زمانی که finished در خروجی تابع step برابر با یک شد، decoding برای آن نمونه به اتمام رسیده است. در این زمان تابع finalize دیکودر با ورودی های مذکور فراخوانده می شود تا خروجی و state نهایی decoding را مشخص کند.
        خروجی این تابع می بایست به صورت (final_outputs, final_state) باشد.
        <br/>
        نکته ی مهم این است که برخی از decoder هایی که خود tensorflow نیز پیاده سازی کرده این تابع را ندارند و در استفاده از آنها مشکلی نیز پیش نمی آید. علت این است که تابع dynamic_decode در بلوک try-catch قرار داده شده و اگر دیکودر مورد استفاده ی dynamic_decode تابع finalize را پیاده سازی نکرده بود، state آخرین مرحله ی decoding به عنوان final_state و یک tensoerArray به صورتی که درایه ی iاُم آن خروجی مرحله ی iاُم decoding است به عنوان final_outputs در خروجی dynamic_decode قرار می گیرد.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        <b>دیکودرهای تنسورفلو</b>
        <br/>
        اصول دیکودرها و dynamic decoding در tensorflow مطالب گفته شده در بالا است. tensorflow با نوشتن دو دیکودر کار ما را ساده تر کرده است. این دو دیکودر BasicDecoder و BeamSearchDecoder اند.
        <br/>
        حال به صورت جداگانه این دو را بررسی می کنیم.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        <b>BasicDecoder</b>
        <br/>
        شاید پراستفاده ترین decodingی که مورد استفاده قرار می گیرد به این صورت است که مشابه RNN ورودی و state مرحله ی قبل به یک RNN cell خروجی آن مرحله را  را برای آن مرحله از decoding به دست آوریم.
        <br/>
        تعریف تابع initializer این کلاس را در زیر مشاهده می کنید.
    </font> 
</div>

In [11]:
def __init__(self, cell, helper, initial_state, output_layer=None):
    """Initialize BasicDecoder.

    Args:
      cell: An `RNNCell` instance.
      helper: A `Helper` instance.
      initial_state: A (possibly nested tuple of...) tensors and TensorArrays.
        The initial state of the RNNCell.
      output_layer: (Optional) An instance of `tf.layers.Layer`, i.e.,
        `tf.layers.Dense`. Optional layer to apply to the RNN output prior
        to storing the result or sampling.

    Raises:
      TypeError: if `cell`, `helper` or `output_layer` have an incorrect type.
    """
    pass

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        تابع finalize در BasicDecoder پیاده سازی نشده است.
        <br/>
        در توابع initialize و step این دیکودر از کلاسی به اسم <b>helper</b> استفاده می شود. helper کلاسی است که قابلیت استفاده از BasicDecoder را چنانکه در ادامه توضیح می دهیم افزایش می دهد. عملکرد دو تابع مذکور بدین صورت است که با استفاده از توابعی در helper خروجی های مد نظر را تولید می کند. در واقع helper در تولید خروجی ها به توابع BasicDecoder کمک می کند.
        <br/>
        کلاس Helper کلاسی است که هر helperی باید از توابع آن را implement کند. thank's tensorflow کلاسی داریم به نام CustomHelper! این کلاس با ورودی گرفتن سه تابع اصلی helper، یک instance از Helper به ما خواهد داد. این سه تابع اصلی موارد زیرند:
    </font> 
</div>

In [12]:
import six
@six.add_metaclass(abc.ABCMeta)
class Helper(object):
    """Interface for implementing sampling in seq2seq decoders.

    Helper instances are used by `BasicDecoder`.
    """
    @abc.abstractmethod
    def initialize(self, name=None):
        """Returns `(initial_finished, initial_inputs)`."""
        pass
    
    @abc.abstractmethod
    def sample(self, time, outputs, state, name=None):
        """Returns `sample_ids`."""
        pass
    
    @abc.abstractmethod
    def next_inputs(self, time, outputs, state, sample_ids, name=None):
        """Returns `(finished, next_inputs, next_state)`."""
        pass

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        کافی است این سه تابع را پیاده سازی کرده و به عنوان ورودی به ترتیب بالا به کلاس CustomHelper بدهیم تا یک Helper داشته باشیم:)
        <br/>
        initializer این کلاس را در زیر مشاهده می کنید:
    </font> 
</div>

In [14]:
def __init__(self, initialize_fn, sample_fn, next_inputs_fn,
               sample_ids_shape=None, sample_ids_dtype=None):
    """Initializer.

    Args:
      initialize_fn: callable that returns `(finished, next_inputs)`
        for the first iteration.
      sample_fn: callable that takes `(time, outputs, state)`
        and emits tensor `sample_ids`.
      next_inputs_fn: callable that takes `(time, outputs, state, sample_ids)`
        and emits `(finished, next_inputs, next_state)`.
      sample_ids_shape: Either a list of integers, or a 1-D Tensor of type
        `int32`, the shape of each value in the `sample_ids` batch. Defaults to
        a scalar.
      sample_ids_dtype: The dtype of the `sample_ids` tensor. Defaults to int32.
    """
    self._initialize_fn = initialize_fn
    self._sample_fn = sample_fn
    self._next_inputs_fn = next_inputs_fn
    self._batch_size = None
    self._sample_ids_shape = tensor_shape.TensorShape(sample_ids_shape or [])
    self._sample_ids_dtype = sample_ids_dtype or dtypes.int32


<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        نقش سه تابع ذکر شده را با بررسی استفاده ی آنها در initialize و step کلاس BasicDecoder بیشتر بررسی می کنیم.
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        initialize:<br/>
        تابع initialize در BasicDecoder دو خروجی اول خود یعنی finished و initial_input را از helper می گیرد و initial_State را نیز برابر مقداری قرار می دهد که به همین نام در initializer خود BasicDecoder گرفته شده است.
    </font> 
</div>

In [1]:
def initialize(self, name=None):
    """Initialize the decoder.

    Args:
      name: Name scope for any created operations.

    Returns:
      `(finished, first_inputs, initial_state)`.
    """
    return self._helper.initialize() + (self._initial_state,)

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        next_inputs:<br/>
        همانطور که در <a href="#decoder_output">اینجا</a> ذکر شد تابع step چهار خروجی دارد. BasicDecoder 3 خروجی آخر یعنی (finished, next_inputs, next_state) را با صدا زدن next_inputs در helper به دست می آورد.<br/>
        یعنی می توانیم این سه را با نوشتن تابع next_inputs مد نظرمان و سپس ساخت یک CustomHelper به طور مستقیم کنترل کنبم
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        sample:<br/>
        outputs در خروجی step یک BasicDecoder زوج مرتبی از دو چیز است: (cell_outputs, sample_ids). cell_outputs خروجی RNN cellی است که در ورودی BasicDecoder داده می شود. smaple_ids، دیگر قسمت outputs از تابع sample در helper گرفته می شود.<br/>
        در واقع تابع sample جایی است که ما می توانیم خروجی مورد نظرمان را تولید کنیم. ورودی های این تابع خروجی های RNN cell است.
    </font> 
</div>

In [1]:
def step(self, time, inputs, state, name=None):
    """Perform a decoding step.

    Args:
    time: scalar `int32` tensor.
    inputs: A (structure of) input tensors.
    state: A (structure of) state tensors and TensorArrays.
    name: Name scope for any created operations.

    Returns:
    `(outputs, next_state, next_inputs, finished)`.
    """
    with ops.name_scope(name, "BasicDecoderStep", (time, inputs, state)):
        cell_outputs, cell_state = self._cell(inputs, state)
        if self._output_layer is not None:
            cell_outputs = self._output_layer(cell_outputs)
        sample_ids = self._helper.sample(
            time=time, o utputs=cell_outputs, state=cell_state)
        (finished, next_inputs, next_state) = self._helper.next_inputs(
            time=time,
            outputs=cell_outputs,
            state=cell_state,
            sample_ids=sample_ids)
        outputs = BasicDecoderOutput(cell_outputs, sample_ids)
        return (outputs, next_state, next_inputs, finished)


SyntaxError: invalid syntax (<ipython-input-1-83d19eccd21f>, line 18)

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        <b>BeamSearchDecoder</b><br/>
        دیکودر دیگری که در tensorflow پیاده سازی شده BeamSearchDecoder است. تابع initializer این دیکودر به صورت زیر می باشد:
    </font> 
</div>

In [2]:
def __init__(self,
    cell,
    embedding,
    start_tokens,
    end_token,
    initial_state,
    beam_width,
    output_layer=None,
    length_penalty_weight=0.0,
    coverage_penalty_weight=0.0,
    reorder_tensor_arrays=True):
    """Initialize the BeamSearchDecoder.

    Args:
      cell: An `RNNCell` instance.
      embedding: A callable that takes a vector tensor of `ids` (argmax ids),
        or the `params` argument for `embedding_lookup`.
      start_tokens: `int32` vector shaped `[batch_size]`, the start tokens.
      end_token: `int32` scalar, the token that marks end of decoding.
      initial_state: A (possibly nested tuple of...) tensors and TensorArrays.
      beam_width:  Python integer, the number of beams.
      output_layer: (Optional) An instance of `tf.layers.Layer`, i.e.,
        `tf.layers.Dense`.  Optional layer to apply to the RNN output prior
        to storing the result or sampling.
      length_penalty_weight: Float weight to penalize length. Disabled with 0.0.
      coverage_penalty_weight: Float weight to penalize the coverage of source
        sentence. Disabled with 0.0.
      reorder_tensor_arrays: If `True`, `TensorArray`s' elements within the cell
        state will be reordered according to the beam search path. If the
        `TensorArray` can be reordered, the stacked form will be returned.
        Otherwise, the `TensorArray` will be returned as is. Set this flag to
        `False` if the cell state contains `TensorArray`s that are not amenable
        to reordering.

    Raises:
      TypeError: if `cell` is not an instance of `RNNCell`,
        or `output_layer` is not an instance of `tf.layers.Layer`.
      ValueError: If `start_tokens` is not a vector or
        `end_token` is not a scalar.
    """
    pass

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        initial_state به عنوان state اولیه به cell داده می شود. state در خروجی تابع initialize و step یک instance از BeamSearchDecoderState است. BeamSearchDecoderState شامل موارد زیر است:<br/>
        <ul style="direction:ltr">
            <li>cell_state</li>
            <li>log_probs</li>
            <li>finished</li>
            <li>lengths</li>
            <li>accumulated_attention_probs</li>
        </ul>
        
    </font> 
</div>

<div style="direction:rtl;line-height:300%;">
    <font face="XB Zar" size=5>
        خروجی initial_inputs در تابع initialize این دیکودر برابر (embedding(start_tokens است. 
    </font> 
</div>